In [1]:
# Importing the relevant packages
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime  # use it for LOG



## Downloading and preprocessing the data

In [2]:

BUFFER_SIZE = 70_000 # same as is
BATCH_SIZE = 128
NUM_EPOCHS = 5

In [3]:
# Downloading the MNIST dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [5]:
# Creating a function to scale our data
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.

    return image, label

In [6]:
# Scaling the data
train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [7]:
# size of validation set
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [8]:
# Defining size of test set
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [9]:
# Reshuffle
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [10]:
# Splitting the dataset 
train_data = train_and_validation_data.skip(num_validation_samples)
validation_data = train_and_validation_data.take(num_validation_samples)

In [11]:
# Batch
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

## Creating the model and training it

In [12]:
# SAME

In [13]:

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50, 5, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(50, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

In [14]:
model.summary(line_length = 75)

Model: "sequential"
___________________________________________________________________________
 Layer (type)                    Output Shape                  Param #     
 conv2d (Conv2D)                 (None, 24, 24, 50)            1300        
                                                                           
 max_pooling2d (MaxPooling2D)    (None, 12, 12, 50)            0           
                                                                           
 conv2d_1 (Conv2D)               (None, 10, 10, 50)            22550       
                                                                           
 max_pooling2d_1 (MaxPooling2D)  (None, 5, 5, 50)              0           
                                                                           
 flatten (Flatten)               (None, 1250)                  0           
                                                                           
 dense (Dense)                   (None, 10)                    12510

In [15]:
# loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [16]:
#  Adam optimizer 
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [17]:
# prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0, 
    restore_best_weights = True
)

In [18]:
# Logging the training process data to use later in tensorboard
# we run our model with different parameters thats why datetime ( we have each iteration and each trial)
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#above my parameter for training

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
#callback is a function or an operation that is executed at the end of each epoch
#now we have tensorboard as callback

In [19]:
# Train the network
model.fit(
    train_data, 
    epochs = NUM_EPOCHS, 
    callbacks = [tensorboard_callback, early_stopping],  #2 callbacks earlystop and tfBOARD
    validation_data = validation_data,
    verbose = 2   # now training is recorded
)

Epoch 1/5
422/422 - 30s - loss: 0.2648 - accuracy: 0.9239 - val_loss: 0.0760 - val_accuracy: 0.9773 - 30s/epoch - 72ms/step
Epoch 2/5
422/422 - 27s - loss: 0.0677 - accuracy: 0.9799 - val_loss: 0.0591 - val_accuracy: 0.9815 - 27s/epoch - 63ms/step
Epoch 3/5
422/422 - 25s - loss: 0.0495 - accuracy: 0.9852 - val_loss: 0.0397 - val_accuracy: 0.9875 - 25s/epoch - 60ms/step
Epoch 4/5
422/422 - 25s - loss: 0.0398 - accuracy: 0.9879 - val_loss: 0.0302 - val_accuracy: 0.9905 - 25s/epoch - 59ms/step
Epoch 5/5
422/422 - 22s - loss: 0.0342 - accuracy: 0.9896 - val_loss: 0.0247 - val_accuracy: 0.9923 - 22s/epoch - 52ms/step


## Testing your HOT content

In [20]:
# Testing our model
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.0282 - accuracy: 0.9910


In [21]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.0282. Test accuracy: 99.10%


## Visualizing in Tensorboard

In [22]:
# Loading the Tensorboard extension   # use it for visualization  #my logs for this training!
%load_ext tensorboard
%tensorboard --logdir "logs/fit" 
